### 크롤링 및 전처리

In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from bs4 import BeautifulSoup
import random, time
from tqdm import tqdm
import pandas as pd

In [4]:
col = ['species', 'number', 'anothername', 'scientific', 'herbal', 'possible', 'limit', 'part']
df = pd.DataFrame(columns=col)
for n in range(1,12):
    print(n)
    # URL
    html = f'https://www.foodsafetykorea.go.kr/portal/safefoodlife/foodMeterial/foodMeterialDB.do?menu_no=3090&menu_grp=MENU_NEW07&search_type=&animal=%EB%AF%B8%EC%83%9D%EB%AC%BC&srch_name=&edible=&page={n}'
    # 텍스트로 바꿔준다
    web = urlopen(html)
    soup = BeautifulSoup(web)
    # td와 tl만 찾음 (원재료명 코드)
    species = soup.find_all('td', class_='tl')
    # 텍스트만 (종이름만 추출)
    text = [td.get_text() for td in species]
    # 그 뒤의 td에 남은 추가 설명을 리스트화
    find_list = soup.find_all("td")
    find = [f.get_text() for f in find_list]
    # OX를 추가하면서 수가 안맞아짐 추후에 추가하거나 할 것 (ex: '제한'의 -는 X로 바꾸고, '제한'에 X가 있으면 '가능'은 O으로 바꾸고 )
    find = [item for item in find if 'O' not in item and 'X' not in item]
    # 8개 단위로 자르기
    microbes = [find[i:i+8] for i in range(0, len(find), 8)]

    # 결과 출력
    for microbe in microbes:
        # print(microbe)
        new_row = pd.Series(microbe, index=col)
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)


    # 필요한 부분만 선택하기
    # selected_microbes = [microbe for microbe in microbes if microbe[0] != '-']
print(df)


        




1
2
3
4
5
6
7
8
9
10
11
                      species number  \
0           Acetobacter aceti      -   
1    Acetobacter pasteurianus      -   
2         Acetobacter xylinum      -   
3    Arthrobacter globiformis      -   
4       Aspergillus luchensis      -   
..                        ...    ...   
103         Weissella cibaria      -   
104         Weissella confusa      -   
105       Weissella koreensis      -   
106  Zygosaccharomyces rouxii      -   
107         Zymomonas mobilis      -   

                                           anothername  \
0                       Acetobacter aceti subsp. aceti   
1    Acetobacter pasteurianus subsp. ascendens, Ace...   
2    Acetobacter aceti subsp. xylinum, Acetobacter ...   
3    Achromobacter globiformis, Bacterium globiform...   
4            Aspergillus kawachii, Aspergillus awamori   
..                                                 ...   
103                                                  -   
104  Lactobacillus confusus, La

In [5]:
df['limit'] = df['limit'].replace('-', 'X')

df.loc[df['limit'] == 'X', 'possible'] = df['possible'].replace('-', 'O')


In [6]:
del df['number']
del df['herbal']
del df['part']


In [7]:
df

,species,anothername,scientific,possible,limit
0,Acetobacter aceti,Acetobacter aceti subsp. aceti,Acetobacter aceti,-,식초(초산발효 포함) 제조에 한함
1,Acetobacter pasteurianus,"Acetobacter pasteurianus subsp. ascendens, Ace...",Acetobacte rpasteurianus,-,식초(초산발효 포함) 제조에 한함
2,Acetobacter xylinum,"Acetobacter aceti subsp. xylinum, Acetobacter ...",Acetobacter xylinum,-,바이오셀룰로즈 제조에 한함
3,Arthrobacter globiformis,"Achromobacter globiformis, Bacterium globiform...",Arthrobacter globiformis,-,치즈 제조에 한함
4,Aspergillus luchensis,"Aspergillus kawachii, Aspergillus awamori",Aspergillus luchensis,O,X
...,...,...,...,...,...
103,Weissella cibaria,-,Weissella cibaria,O,X
104,Weissella confusa,"Lactobacillus confusus, Lactobacillus coprophi...",Weissella confusa,O,X
105,Weissella koreensis,-,Weissella koreensis,O,X
106,Zygosaccharomyces rouxii,Kluyveromyces osmophilus,Zygosaccharomyces rouxii,-,"장류 제조 (간장, 된장)"


In [19]:
df['possible'].unique()

array(['-', 'O', 'Lactiplantibacillus plantarum CNCM MA40/5B-p는 제외',
       'Pediococcus acidilactici CNCM MA28/6B는 제외',
       'Stereptococcus salivarius AD601는 제외'], dtype=object)

### MariaDB에 저장

In [22]:
import mariadb

# 데이터베이스 연결 설정
conn = mariadb.connect(
    host='127.0.0.1',  # 데이터베이스 호스트
    user='root',  # 사용자 이름
    password='root',  # 비밀번호
    database='micro_organism'  # 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()

query = "ALTER TABLE microbes MODIFY COLUMN possible VARCHAR(100);" 

# 데이터프레임(df)에서 데이터를 반복해서 데이터베이스에 삽입
for index, row in df.iterrows():
    species = row['species']
    another_name = row['anothername']
    scientific_name = row['scientific']
    possible = row['possible']
    limit_ = row['limit']
    
    # 데이터베이스에 데이터 삽입
    query = "INSERT INTO microbes (species, another_name, scientific_name, possible, limit_) VALUES (%s, %s, %s, %s, %s)"
    data = (species, another_name, scientific_name, possible, limit_)
    cursor.execute(query, data)

# 커밋 (변경사항 저장)
conn.commit()

# 연결 종료
conn.close()

# data = (f"{df['speices']}, {df['anothername']}, {df['scientificname']}, {df['possible']}, {df['limit']}")



### EXCEL 파일로 저장

In [8]:
df.to_excel("microbes_food.xlsx", index=False)


### 진행과정

In [ ]:
species = soup.find_all('td', class_='tl')

In [ ]:
text = [td.get_text() for td in species]
print(text)


In [ ]:
# find_list = soup.find_all("td", class_="tl")
find_list = soup.find_all("td")
find_list

In [ ]:
# 원재료명 / 품목번호 / 영명 / 학명 / 생약명 / "식용가능" / 사용제한 / 사용부위 / 사용조건 / 비고    13을 주기로 인덱스 2,3,4,5
find = [f.get_text() for f in find_list]
print(find)

# col = ['species', '']

In [ ]:
find = [item for item in find if 'O' not in item and 'X' not in item]
find

In [ ]:
# 8개 단위로 자르기
microbes = [find[i:i+8] for i in range(0, len(find), 8)]

# 필요한 부분만 선택하기
# selected_microbes = [microbe for microbe in microbes if microbe[0] != '-']

# 결과 출력
for microbe in microbes:
    print(microbe)

['Bacillus natto', '-', 'Bacillus subtilis', 'Bacillus natto', '-', '-', '-', '-']
['Bacillus polyfermenticus', '-', '-', 'Bacillus polyfermenticus', '-', '-', '-', '-']
['Bacillus subtilis', '-', 'Bacillus globigii, Bacillus natto', 'Bacillus subtilis', '-', '-', '-', '-']
['Bifidobacterium adolescentis', '-', '-', 'Bifidobacterium adolescentis', '-', '-', '-', '-']
['Bifidobacterium animalis', '-', '-', 'Bifidobacterium animalis subsp. animalis', '-', '-', '-', '-']
['Bifidobacterium bifidum', '-', 'Actinomyces parabifidus', 'Bifidobacterium bifidum', '-', '-', '-', '-']
['Bifidobacterium breve', '-', 'Bifidobacterium parvulorum', 'Bifidobacterium breve', '-', '-', '-', '-']
['Bifidobacterium infantis', '-', '-', 'Bifidobacterium longum subsp. infantis', '-', '-', '-', '-']
['Bifidobacterium lactis', '-', '-', 'Bifidobacterium animalis subsp. lactis', '-', '-', '-', '-']
['Bifidobacterium longum', '-', '-', 'Bifidobacterium longum subsp. longum', '-', '-', '-', '-']
